# Imports

In [ ]:
import numpy as np 
import pandas as pd
import datatable as dt
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

# Load dataset

In [ ]:
%%time
train = dt.fread('../input/tabular-playground-series-oct-2021/train.csv').to_pandas()
test  = dt.fread('../input/tabular-playground-series-oct-2021/test.csv').to_pandas()
sub   = dt.fread('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()

In [ ]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64','float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                else:
                    df[col] = df[col].astype(np.float32)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
test  = reduce_mem_usage(test)

In [ ]:
y = train['target']
train.drop('target',axis=1,inplace=True)

# Model hyperparameters

In [ ]:
# lgbm params
lgbm_params = {
  "objective": "binary",
  "metric": "auc",
  "learning_rate": 0.08,
  "device": "gpu",
  "verbose": 0, 
  "feature_pre_filter": False, 
  "lambda_l1": 9.314037635261775, 
  "lambda_l2": 0.10613573572440353,
  "num_leaves": 7,
  "feature_fraction": 0.4, 
  "bagging_fraction": 0.8391963650875751, 
  "bagging_freq": 5, 
  "min_child_samples": 100,
  "num_iterations": 10000,
  "n_estimators": 20000,
  "random_state": 42
}

# 5-fold model training and prediction

In [ ]:
folds = KFold(n_splits = 5, random_state = 102021, shuffle = True)

predictions = np.zeros(len(test))
lgbm_oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(folds.split(train)):

    X_train, X_test = train.iloc[trn_idx], train.iloc[val_idx]
    y_train, y_test = y.iloc[trn_idx], y.iloc[val_idx]

    model = LGBMClassifier(**lgbm_params)
    print(f"model fit started for Fold: {fold}")
    model.fit(X_train, 
              y_train,
              eval_set=[(X_test, y_test)],
              early_stopping_rounds=400,
              verbose=False)
    pred = model.predict_proba(X_test)[:,1]
    lgbm_oof[val_idx] = pred
    roc = roc_auc_score(y_test, pred)
    print(f" roc_auc_score: {roc}")
    print("-"*50)
    
    predictions += model.predict_proba(test)[:,1] / folds.n_splits 

# Store submission and oof 

In [ ]:
sub['target'] = predictions
sub.to_csv(f'submission_lgbm_1.csv',index = False)
np.savez_compressed('oof_lgbm.npz', lgbm_oof)